# Модуль для эксперементов с полями товара Алиехпресс

## Стартовые установки и импорты

In [ ]:
import sys
import os
from pathlib import Path

# Добавляю к переменной путей в ОС (sys.path) путь к корню проекта и путь к файлам программы
dir_root : Path = Path (os.getcwd()[:os.getcwd().rfind('hypotez')+8])
print (dir_root)
sys.path.append (str (dir_root) )  # Добавляю корневую папку в sys.path
dir_src = Path (dir_root, 'src')
print (dir_src)
sys.path.append (str (dir_src) )  # Добавляю рабочую папку в sys.path 


from suppliers import Supplier
from product import Product, ProductFields
from tools import StringFormatter, StringNormalizer, StringValidator as SV
from helpers import logger, pprint, jprint

## Поставщик
### С поставщиком загрузится и его вебдрайвер. Открытие окна браузера займет некоторое время

In [ ]:
def start_supplier(supplier_prefix: str = 'aliexpress', locale: str = 'en' ):
    """ Старт поставщика """
    params: dict = \
    {
        'supplier_prefix': supplier_prefix,
        'locale': locale
    }
    
    return Supplier(**params)
s = start_supplier('aliexpress')

## тестовыe сценарии и набор URL

In [ ]:
test_scenario: dict = {
  "category ID on site": 40000002781737,
  "brand": "APPLE",
  "url": "https://hi5group.aliexpress.com/store/group/iPhone-13-13-mini/1053035_40000002781737.html",
  "active": True,
  "condition": "new",
  "presta_categories": {
    "template": {
      "apple": "iPhone 13"
    },
      "default_category": { "11209": "MURANO GLASS" }
  },
  
  "product combinations": [
    "bundle",
    "color"
  ]
}

test_products_list: list = ['https://s.click.aliexpress.com/e/_oFLpkfz', 
                        'https://s.click.aliexpress.com/e/_oE5V3d9', 
                        'https://s.click.aliexpress.com/e/_oDnvttN', 
                        'https://s.click.aliexpress.com/e/_olWWQCP', 
                        'https://s.click.aliexpress.com/e/_ok0xeMn']

### Установки для работы с товаром.

- `p` - инстанс класса `Product` <br/>
- `_l` - словарь локаторов вебэлементов. <br/>
Локаторы настроены каждый на свой элемент страницы `html`. Вебдрайвер `Driver` через локаторы исследует целевую страницу<br>
- `_d` инстанс класса `Driver`, который подключен к этому поставщику
- `_f` инстанс класса `ProductFields`, который принимая значения от `_d` переводит его в словарь полей формата `API PrestaShop`

In [ ]:
s.current_scenario = test_scenario
p = Product(supplier = s)
_l : dict = s.locators['product']
_d = s.driver
_f = p.product_fields

#### перехожу по URL

In [ ]:
_d.get_url(test_products_list[0])

#### собираю поля

In [ ]:
_f.supplier_reference = _d.current_url.split('/')[-1].split('.')[0]
_f.reference = f'''{s.supplier_id}-{_f.supplier_reference}'''
#_f.name = StringNormalizer.normalize_product_name ( _d.execute_locator( _l['name'] )[0])
_f.name =  _d.execute_locator( _l['name'] )[0]

_category_default = list(s.current_scenario['presta_categories']['default_category'].keys())[0]
_f.id_category_default = _category_default

_f.id_lang
_f.id_manufacturer = s.current_scenario['brand']
_f.id_shop_default
_f.id_supplier = s.supplier_id
_f.id_tax = 13
_f.affiliate_short_link = _d.execute_locator (_l['affiliate_short_link'])


In [ ]:
pprint( _f.name['language'][0]['value'] )

In [ ]:
pprint ( _f.id_category_default)